In [1]:
import nltk
from nltk.chunk import tree2conlltags #구로 묶는 기능--tree형태 태그만들기
from nltk.corpus import names #성별 확인용
import random

In [2]:
def feature(word):
    return {'last(1)' : word[-1]} #단순히 마지막 글자로 구분

In [3]:
males = [(name, 'male') for name in names.words('male.txt')]
females = [(name, 'female') for name in names.words('female.txt')]
combined = males + females #리스트 병합
random.shuffle(combined) #뒤섞기
training = [(feature(name), gender) for (name, gender) in combined] #이름의 마지막 글자와 성별이 training data로 담김

In [4]:
males[:10]

[('Aamir', 'male'),
 ('Aaron', 'male'),
 ('Abbey', 'male'),
 ('Abbie', 'male'),
 ('Abbot', 'male'),
 ('Abbott', 'male'),
 ('Abby', 'male'),
 ('Abdel', 'male'),
 ('Abdul', 'male'),
 ('Abdulkarim', 'male')]

In [5]:
females[:10]

[('Abagael', 'female'),
 ('Abagail', 'female'),
 ('Abbe', 'female'),
 ('Abbey', 'female'),
 ('Abbi', 'female'),
 ('Abbie', 'female'),
 ('Abby', 'female'),
 ('Abigael', 'female'),
 ('Abigail', 'female'),
 ('Abigale', 'female')]

In [6]:
combined[:10]

[('Ida', 'female'),
 ('Cornelius', 'male'),
 ('Rafaela', 'female'),
 ('Roseanna', 'female'),
 ('Lizabeth', 'female'),
 ('Van', 'female'),
 ('Trevar', 'male'),
 ('Tonnie', 'male'),
 ('Zsa Zsa', 'female'),
 ('Margareta', 'female')]

In [7]:
training[:10]

[({'last(1)': 'a'}, 'female'),
 ({'last(1)': 's'}, 'male'),
 ({'last(1)': 'a'}, 'female'),
 ({'last(1)': 'a'}, 'female'),
 ({'last(1)': 'h'}, 'female'),
 ({'last(1)': 'n'}, 'female'),
 ({'last(1)': 'r'}, 'male'),
 ({'last(1)': 'e'}, 'male'),
 ({'last(1)': 'a'}, 'female'),
 ({'last(1)': 'a'}, 'female')]

In [8]:
classifier = nltk.NaiveBayesClassifier.train(training)

In [9]:
sentences = [
    "John is a man. He walks",
    "John and Mary are married. They have two kids",
    "In order for Ravi to be successful, he should follow John",
    "John met Mary in Barista. She asked him to order a Pizza"
]

In [10]:
def gender(word):
    return classifier.classify(feature(word))

In [11]:
for sent in sentences:
    chunks = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent)), binary=False) #토큰 분리, 품사 태깅, 청킹/ named ntt로 표기, 사람이름 person으로 표기!
    stack = []
    print(sent)
    items = tree2conlltags(chunks) #tree->tag 폼 변환
    for item in items:
        if item[1] == 'NNP' and (item[2] == 'B-PERSON' or item[2] == 'O'): #고유명사
            stack.append((item[0], gender(item[0]))) #stack이라는 변수에 John, male 을 넣어줌
        elif item[1] == 'CC': #등위접속사 John and Mary 처럼 이름 뒤 이름이 발로 나올 확률
            stack.append(item[0])
        elif item[1] == 'PRP':# 대명사
            stack.append(item[0])
    print("\t {}".format(stack))

John is a man. He walks
	 [('John', 'male'), 'He']
John and Mary are married. They have two kids
	 [('John', 'male'), 'and', ('Mary', 'female'), 'They']
In order for Ravi to be successful, he should follow John
	 [('Ravi', 'female'), 'he', ('John', 'male')]
John met Mary in Barista. She asked him to order a Pizza
	 [('John', 'male'), ('Mary', 'female'), 'She', 'him']


In [ ]:
he는 john이다, male이다 이런식으로 찾아줌

In [12]:
items

[('John', 'NNP', 'B-PERSON'),
 ('met', 'VBD', 'O'),
 ('Mary', 'NNP', 'O'),
 ('in', 'IN', 'O'),
 ('Barista', 'NNP', 'B-GPE'),
 ('.', '.', 'O'),
 ('She', 'PRP', 'O'),
 ('asked', 'VBD', 'O'),
 ('him', 'PRP', 'O'),
 ('to', 'TO', 'O'),
 ('order', 'NN', 'O'),
 ('a', 'DT', 'O'),
 ('Pizza', 'NN', 'O')]

In [13]:
print(chunks)

(S
  (PERSON John/NNP)
  met/VBD
  Mary/NNP
  in/IN
  (GPE Barista/NNP)
  ./.
  She/PRP
  asked/VBD
  him/PRP
  to/TO
  order/NN
  a/DT
  Pizza/NN)
